In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
file_path = "Final_Training_Data.csv"  
df = pd.read_csv(file_path)
print(df)

      age gender residence  competative_exams  access_to_internet  \
0      11      M         H                  0                   0   
1      14      F         D                  0                   1   
2      21      M         H                  1                   1   
3      24      F         H                  1                   0   
4      12      F         H                  0                   0   
...   ...    ...       ...                ...                 ...   
2054   18      F         H                  1                   1   
2055   20      M         H                  0                   0   
2056   26      F         H                  1                   0   
2057   27      M         D                  1                   0   
2058   18      F         D                  1                   0   

      study_hours_per_day  has_scholarship  extracurricular_activities  \
0                       4                0                           1   
1                      

In [3]:
df_encoded = pd.get_dummies(df,columns=["gender","residence","location"])

In [4]:
df_encoded.shape

(2059, 26)

In [5]:
df_encoded.columns

Index(['age', 'competative_exams', 'access_to_internet', 'study_hours_per_day',
       'has_scholarship', 'extracurricular_activities', 'fam_fin_status',
       'health', 'f_edu', 'm_edu', 'f_job', 'm_job', 'alcoholic',
       'additional_tuition', 'fam_size', 'fam_relation', 'prev_percent_1',
       'prev_percent_2', 'prev_percent_3', 'TARGET_PREDICTION_PERCENT',
       'gender_F', 'gender_M', 'residence_D', 'residence_H', 'location_R',
       'location_U'],
      dtype='object')

In [6]:
# Features are all columns except the target
X = df_encoded.drop("TARGET_PREDICTION_PERCENT", axis=1)

# Target is the specific output label
y = df_encoded["TARGET_PREDICTION_PERCENT"]


In [7]:
# Display the first few rows of the features and target
print("Features:")
print(X.head())

print("\nTarget:")
print(y.head())


Features:
   age  competative_exams  access_to_internet  study_hours_per_day  \
0   11                  0                   0                    4   
1   14                  0                   1                    1   
2   21                  1                   1                    6   
3   24                  1                   0                    7   
4   12                  0                   0                    5   

   has_scholarship  extracurricular_activities  fam_fin_status  health  f_edu  \
0                0                           1               3       5      4   
1                1                           0               4       2      5   
2                0                           1               5       3      3   
3                0                           0               4       2      2   
4                1                           0               3       1      3   

   m_edu  ...  fam_relation  prev_percent_1  prev_percent_2  prev_percent_3  \
0  

In [8]:

# Split the data into training and testing sets
# 80% for training and 20% for testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify the size of the splits
print("Training Features shape:", X_train.shape)
print("Testing Features shape:", X_test.shape)
print("Training Target shape:", y_train.shape)
print("Testing Target shape:", y_test.shape)

Training Features shape: (1647, 25)
Testing Features shape: (412, 25)
Training Target shape: (1647,)
Testing Target shape: (412,)


In [9]:
scaler=StandardScaler()

In [10]:
X_train = scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

In [11]:
X_test

array([[ 0.46366033, -0.91165296, -2.02148949, ..., -0.94795438,
         1.17537351, -1.17537351],
       [ 0.77630854,  1.09690864, -2.02148949, ...,  1.05490308,
        -0.85079338,  0.85079338],
       [ 1.08895675, -0.91165296, -2.02148949, ..., -0.94795438,
         1.17537351, -1.17537351],
       ...,
       [-0.16163609, -0.91165296,  0.49468474, ...,  1.05490308,
        -0.85079338,  0.85079338],
       [-0.16163609, -0.91165296,  0.49468474, ..., -0.94795438,
        -0.85079338,  0.85079338],
       [ 0.15101212,  1.09690864,  0.49468474, ...,  1.05490308,
        -0.85079338,  0.85079338]])

In [12]:

X_train

array([[ 1.45674283, -0.89883447,  0.5024654 , ..., -0.88784774,
        -0.78908814,  0.78908814],
       [-0.70920869, -0.89883447,  0.5024654 , ...,  1.12631924,
         1.26728555, -1.26728555],
       [-0.39978704, -0.89883447,  0.5024654 , ...,  1.12631924,
         1.26728555, -1.26728555],
       ...,
       [-1.32805198,  1.1125519 ,  0.5024654 , ..., -0.88784774,
         1.26728555, -1.26728555],
       [ 0.21905625,  1.1125519 ,  0.5024654 , ...,  1.12631924,
        -0.78908814,  0.78908814],
       [ 0.52847789,  1.1125519 , -1.99018677, ..., -0.88784774,
        -0.78908814,  0.78908814]])

In [13]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(25, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with 25 units and ReLU activation
    tf.keras.layers.Dense(1)  # Output layer with 1 unit (for regression)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae', tf.keras.metrics.RootMeanSquaredError()]  
)

In [14]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10)
mse, rmse,mae = model.evaluate(X_test, y_test, verbose=0)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

Epoch 1/100
165/165 [==============================] - 1s 1ms/step - loss: 6568.9385 - mae: 80.0843 - root_mean_squared_error: 81.0490
Epoch 2/100
165/165 [==============================] - 0s 1ms/step - loss: 6026.8569 - mae: 76.6733 - root_mean_squared_error: 77.6328
Epoch 3/100
165/165 [==============================] - 0s 1ms/step - loss: 5158.9478 - mae: 70.8423 - root_mean_squared_error: 71.8258
Epoch 4/100
165/165 [==============================] - 0s 1ms/step - loss: 3987.0322 - mae: 61.9187 - root_mean_squared_error: 63.1430
Epoch 5/100
165/165 [==============================] - 0s 1ms/step - loss: 2765.0410 - mae: 50.6564 - root_mean_squared_error: 52.5837
Epoch 6/100
165/165 [==============================] - 0s 1ms/step - loss: 1715.5419 - mae: 38.3793 - root_mean_squared_error: 41.4191
Epoch 7/100
165/165 [==============================] - 0s 1ms/step - loss: 962.0319 - mae: 27.2682 - root_mean_squared_error: 31.0166
Epoch 8/100
165/165 [==============================] - 0

In [15]:
#  Load the dataset
data = pd.read_csv("Final_Training_Data.csv")

#  Shuffle and split the data into training and testing subsets (80-20 split)
train, test = train_test_split(data, test_size=0.2, random_state=42)

#  Save the training data to 'train.csv' and testing data to 'test.csv'
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)


In [16]:
test=pd.read_csv("test.csv")

In [17]:
print(test.shape)
test.columns

(412, 23)


Index(['age', 'gender', 'residence', 'competative_exams', 'access_to_internet',
       'study_hours_per_day', 'has_scholarship', 'extracurricular_activities',
       'fam_fin_status', 'health', 'location', 'f_edu', 'm_edu', 'f_job',
       'm_job', 'alcoholic', 'additional_tuition', 'fam_size', 'fam_relation',
       'prev_percent_1', 'prev_percent_2', 'prev_percent_3',
       'TARGET_PREDICTION_PERCENT'],
      dtype='object')

In [18]:
test_encoded = pd.get_dummies(test,columns=["gender","residence","location"])
print(test_encoded.shape)
test_encoded.columns

(412, 26)


Index(['age', 'competative_exams', 'access_to_internet', 'study_hours_per_day',
       'has_scholarship', 'extracurricular_activities', 'fam_fin_status',
       'health', 'f_edu', 'm_edu', 'f_job', 'm_job', 'alcoholic',
       'additional_tuition', 'fam_size', 'fam_relation', 'prev_percent_1',
       'prev_percent_2', 'prev_percent_3', 'TARGET_PREDICTION_PERCENT',
       'gender_F', 'gender_M', 'residence_D', 'residence_H', 'location_R',
       'location_U'],
      dtype='object')

In [19]:
# Features are all columns except the target
test_encoded_x = test_encoded.drop("TARGET_PREDICTION_PERCENT", axis=1)

# Target is the specific output label
test_encoded_y = test_encoded["TARGET_PREDICTION_PERCENT"]


In [20]:
test_transform_x=scaler.fit_transform(test_encoded_x)

In [21]:
test_transform_x

array([[ 0.46366033, -0.91165296, -2.02148949, ..., -0.94795438,
         1.17537351, -1.17537351],
       [ 0.77630854,  1.09690864, -2.02148949, ...,  1.05490308,
        -0.85079338,  0.85079338],
       [ 1.08895675, -0.91165296, -2.02148949, ..., -0.94795438,
         1.17537351, -1.17537351],
       ...,
       [-0.16163609, -0.91165296,  0.49468474, ...,  1.05490308,
        -0.85079338,  0.85079338],
       [-0.16163609, -0.91165296,  0.49468474, ..., -0.94795438,
        -0.85079338,  0.85079338],
       [ 0.15101212,  1.09690864,  0.49468474, ...,  1.05490308,
        -0.85079338,  0.85079338]])

In [22]:
predictions = model.predict(test_transform_x)  

13/13 [==============================] - 0s 1ms/step


In [23]:
test['Predicted_Value'] = predictions

In [24]:
test.to_csv("predictions.csv", index=False)

In [55]:
def create_model(layers=None,activation='relu'):
    if layers is None:
        layers = [128, 64]  # Default layer configuration
    
    model = Sequential()
    model.add(Dense(layers[0], input_dim=X_train.shape[1], activation=activation))  # First layer with input
    for layer_size in layers[1:]:
        model.add(Dense(layer_size, activation=activation))  # Additional hidden layers
    model.add(Dense(1))  # Output layer for regression
     # Output layer with linear activation for regression
    # model.add(layers.Dense(1, activation='linear'))


    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mse'])  # Compile with specified loss, optimizer, metrics
    return model

# Create the Keras regressor
model = KerasRegressor(build_fn=create_model,verbose=1)

# Define the extended hyperparameter grid for GridSearchCV
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adam']
layers = [[25],[128,64]]  # Different layer configurations
# activation = ['relu','linear']  # Activation functions
# loss = ['mean_squared_error', 'mean_absolute_error']  # Loss functions
batch_size=[16, 32, 64]  # Different batch sizes
epochs = [10, 30, 50]  # Different epoch sizes

param_grid = {
#    'activation':activation,
    'layers': layers,
    'batch_size': batch_size,
    'epochs': epochs,
    
}

# Apply GridSearchCV with the extended parameter grid
grid = GridSearchCV(estimator=model, param_grid=param_grid)

grid_result = grid.fit(X_train, y_train)

Epoch 1/10


C:\Users\ASUS\AppData\Local\Temp\ipykernel_14252\87157787.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=create_model,verbose=1)


83/83 [==============================] - 1s 2ms/step - loss: 80.1247 - mse: 6581.0332
Epoch 2/10
83/83 [==============================] - 0s 1ms/step - loss: 78.3366 - mse: 6300.8105
Epoch 3/10
83/83 [==============================] - 0s 1ms/step - loss: 76.1520 - mse: 5967.3574
Epoch 4/10
83/83 [==============================] - 0s 1ms/step - loss: 73.3035 - mse: 5545.6938
Epoch 5/10
83/83 [==============================] - 0s 1ms/step - loss: 69.5857 - mse: 5019.5142
Epoch 6/10
83/83 [==============================] - 0s 1ms/step - loss: 64.8330 - mse: 4389.8135
Epoch 7/10
83/83 [==============================] - 0s 1ms/step - loss: 58.9797 - mse: 3679.1821
Epoch 8/10
83/83 [==============================] - 0s 1ms/step - loss: 52.0405 - mse: 2932.3716
Epoch 9/10
83/83 [==============================] - 0s 1ms/step - loss: 44.2094 - mse: 2206.4277
Epoch 10/10
21/21 [==============================] - 0s 1ms/step - loss: 33.9311 - mse: 1399.9497
Epoch 1/10
83/83 [======================

In [54]:
#  Get the best score and best parameter configuration
best_score = grid_result.best_score_  # Best score obtained (negative due to 'neg_mean_squared_error')
best_params = grid_result.best_params_  # Best parameter combination

# Negate the best score to get the actual MSE value
actual_best_score = -best_score  # Best MSE achieved

# Extract the best parameter components
best_layers = best_params['layers']  # Best layer configuration
# best_activation = best_params['activation']  # Best activation function
best_batch_size=best_params['batch_size']
best_epochs=best_params['epochs']
rmse=np.sqrt(actual_best_score)
# Print the best results
print(f"Best Score (MSE): {actual_best_score}")
print(f"Best Score (RMSE): {rmse}")
print(f"Best Layers: {best_layers}")
print(f"Best Batch Size: {best_batch_size}")
print(f"Best Epochs: {best_epochs}")
# print(f"Best Activation Function: {best_activation}")


Best Score (MSE): 7.061165237426758
Best Score (RMSE): 2.657285313515799
Best Layers: [25]
Best Batch Size: 16
Best Epochs: 50


In [42]:
# Build the model with the best layers
model = tf.keras.Sequential([
    tf.keras.layers.Dense(25, activation='relu', input_shape=(X_train.shape[1],)),  # Input layer with 128 units and ReLU activation
    tf.keras.layers.Dense(25, activation='relu'),  # Hidden layer with 64 units and ReLU activation
    tf.keras.layers.Dense(1)  # Output layer with 1 unit (for regression)
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae', tf.keras.metrics.RootMeanSquaredError()]  
)

# Train the model with the best batch size and epochs
history = model.fit(X_train, y_train, epochs=85, batch_size=16)

# Evaluate the model on the test set
mse, rmse,mae = model.evaluate(X_test, y_test, verbose=0)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

Epoch 1/85
103/103 [==============================] - 1s 1ms/step - loss: 6125.9346 - mae: 77.2304 - root_mean_squared_error: 78.2683
Epoch 2/85
103/103 [==============================] - 0s 1ms/step - loss: 3872.1780 - mae: 60.4436 - root_mean_squared_error: 62.2268
Epoch 3/85
103/103 [==============================] - 0s 1ms/step - loss: 955.1312 - mae: 26.5797 - root_mean_squared_error: 30.9052
Epoch 4/85
103/103 [==============================] - 0s 1ms/step - loss: 282.6634 - mae: 13.2863 - root_mean_squared_error: 16.8126
Epoch 5/85
103/103 [==============================] - 0s 1ms/step - loss: 213.3300 - mae: 11.4150 - root_mean_squared_error: 14.6058
Epoch 6/85
103/103 [==============================] - 0s 1ms/step - loss: 182.9698 - mae: 10.5091 - root_mean_squared_error: 13.5266
Epoch 7/85
103/103 [==============================] - 0s 1ms/step - loss: 164.8950 - mae: 9.8946 - root_mean_squared_error: 12.8411
Epoch 8/85
103/103 [==============================] - 0s 1ms/step - 

In [43]:
predictions = model.predict(test_transform_x)

13/13 [==============================] - 0s 1ms/step


In [44]:
test['Modified_Predicted_Value'] = predictions

In [45]:
test.to_csv("predictions.csv", index=False)